In [ ]:
# this file is to update weights of 3x3 patterns

In [ ]:
from numpy.random import RandomState
import numpy as np
np.set_printoptions(precision=2,suppress=True)

import matplotlib.pyplot as plt
import scipy.optimize as opt
from math import *

from scipy.optimize import basinhopping,minimize,differential_evolution

In [ ]:
# define 3x3 patterns 

zv=[1,1,0,0,1,0,0,1,1]
vv=[1,0,1,1,0,1,0,1,0]
nv=[0,1,0,1,0,1,1,0,1]

def noisy(vector):
    result=[]
    for i in range(0,len(vector)):
        v=vector.copy()
        v[i]=np.logical_not(vector[i]).astype(int)
        #print(i,vector[i],v[i])
        result.append(v)
    return result

zpatterns=([zv]+noisy(zv))
vpatterns=([vv]+noisy(vv))
npatterns=([nv]+noisy(nv))

start=0
end=10
patterns=zpatterns[start:end]+vpatterns[start:end]+npatterns[start:end]
patterns_complete=zpatterns+vpatterns+npatterns

lenth=end-start

In [ ]:
# basis functions, phi_pos, phi_neg
def transf_pos(x,K=2.0,y0=0.001,ym=1.5,n=2):
    result=ym*((x/K)**n)/(1+(x/K)**n)+ym*y0
    return result

def transf_neg(x,K=2.0,y0=0.001,ym=1.5,n=2):
    result=ym/(1+(x/K)**n)+y0*ym
    return result

In [ ]:
# define global varaibles

size=3*3
target_L=0.0
target_H=1.2
patterns=patterns

lenth=int(len(patterns)/3)

target_0=np.array([target_H]*lenth+[target_L]*lenth+[target_L]*lenth)
target_1=np.array([target_L]*lenth+[target_H]*lenth+[target_L]*lenth)
target_2=np.array([target_L]*lenth+[target_L]*lenth+[target_H]*lenth)

target=target_0
#----
AHL_unit_level=33# uM 
transf_p=np.zeros(2)
transf_n=np.zeros(2)

# parameter numbers are approximated from values in Table 1
transf_p[0]=transf_pos(x=0,K=50,y0=0.1,ym=1.0,n=0.45)
transf_p[1]=transf_pos(x=AHL_unit_level,K=50,y0=0.1,ym=1.0,n=0.45)
transf_n[0]=transf_neg(x=0,K=0.252,y0=0.341,ym=735,n=0.67)
transf_n[1]=transf_neg(x=AHL_unit_level,K=0.252,y0=0.341,ym=735,n=0.67)
  
#print(transf_p)
#print(transf_n)

phi_pos=np.array([transf_p]*size)
phi_neg=np.array([transf_n]*size)

In [ ]:
def examine_output(w,P,phi_pos,phi_neg):
    y_n=np.zeros(len(P))
    w_ph_=np.zeros(len(P))

    for n in range(0,len(P)):
        pttn=np.array(P[n])
        
        phi_nw=(1.0+w/np.sqrt(w**2))/2*output_fcn(pttn,phi_pos)+\
            (1.0-w/np.sqrt(w**2))/2*output_fcn(pttn,phi_neg)/w
                
        w_ph_[n]=np.dot(w,phi_nw)/len(w)
        y_n[n]=activate(w_ph_[n])
        
    return y_n,w_ph_

def output_fcn(pttn,fcn):
    
    CHL_levels=np.zeros(len(pttn))
    for i in range(0,len(pttn)):
        CHL_levels[i]=fcn[i][pttn[i]]
        
    return CHL_levels

def activate(x):
    K,y0,ym_x,n= 1.10292919e+03,1.94584842e-02,2.15950963e+04,2.33038211e+00

    result=transf_pos(x.sum(),K=K,y0=y0/1e4,ym=2.0,n=n).sum()
    return result

In [ ]:
def train_step1(target,size,eta,phi_pos,phi_neg):
    rndstate=RandomState(2)
    w=rndstate.random_sample(size)
    print('w: ',w)
    r=size

    w_series=[]
    y_series=[]
    delta_series=[]

    for it in range(0,350):
        y_n=np.zeros(len(patterns))
        g_n=np.zeros(len(patterns))
        delta_err=np.zeros(size)
        phi_n=[]
    
        for n in range(0,len(patterns)):
            pttn=np.array(patterns[n])
            # consider pttn as indexes in vector x, 0 means x[0], 1 means x[1]
        
            phi_nw=(1.0+w/np.sqrt(w**2))/2*output_fcn(pttn,phi_pos)+\
                (1.0-w/np.sqrt(w**2))/2*output_fcn(pttn,phi_neg)/w

            g_n[n]=np.dot(w,phi_nw)/r
            y_n[n]=activate(g_n[n])            
            
            tmp=(y_n[n]-target[n])
            tmp=min(target_H,tmp)

            phi_nw_pos=(1.0+w/np.sqrt(w**2))/2*output_fcn(pttn,phi_pos)                
            delta_err+=tmp*phi_nw_pos
        
        w=w-eta*delta_err 
        y_series.append(y_n)
        w_series.append(w)
        delta_series.append(delta_err)
    
    return w,y_series,w_series,delta_series

In [ ]:
# step2
# basinhopping method

def step2(w0):
    y=np.zeros(len(patterns))

    for n in range(0,len(patterns)):
        pttn=np.array(patterns[n])
        phi_nw=(1.0+w0/np.sqrt(w0**2))/2*output_fcn(pttn,phi_pos)+\
            (1.0-w0/np.sqrt(w0**2))/2*output_fcn(pttn,phi_neg)/w0
        y[n]=activate(np.dot(w0,phi_nw)/size)
        
    product=(2*y-target_H)*(2*target-target_H)
    
    return np.var(product)*10+(target_H**2-np.mean(product))

In [ ]:
# find weights using step2 optimization
w=data_series0['w0']
target=target_0
minimizer_kwargs = {"method": "Nelder-Mead"}
res0=basinhopping(step2,w,minimizer_kwargs=minimizer_kwargs,niter=20)

print(res0.lowest_optimization_result.success)
w0_txt=", ".join("%.3f" % f for f in res0.x)
print(w0_txt)

#----
w=data_series0['w1']
target=target_1
minimizer_kwargs = {"method": "Nelder-Mead"}
res1=basinhopping(step2,w,minimizer_kwargs=minimizer_kwargs,niter=20)

print(res1.lowest_optimization_result.success)
w1_txt=", ".join("%.3f" % f for f in res1.x)
print(w1_txt)

#---
w=data_series0['w2']
target=target_2
minimizer_kwargs = {"method": "Nelder-Mead"}
res2=basinhopping(step2,w,minimizer_kwargs=minimizer_kwargs,niter=20)

print(res2.lowest_optimization_result.success)
w2_txt=", ".join("%.3f" % f for f in res2.x)
print(w2_txt)